In [1]:
import numpy as np
from qutip import *
import matplotlib.pyplot as plt
import multiprocessing as mp


# 接地Xmon

## 独立接地比特

### 通过电容和电阻计算比特频率

In [64]:
# 
from QubitEnergyFineSimulation import XmonEnergy
Cd =105.11e-15+9.8e-15
R=7000
ng = 0
elementParameter = [Cd,R,ng]
Xmon1 = XmonEnergy(elementParameter)
el = Xmon1.energyLevel
print(el[1]-el[0])
print(el[2]-el[0]-2*el[1])
print(Xmon1.Ej/2/np.pi)

5.001541002106542
-0.18315536845599212
19.867340604670083


### 计算不同ng下比特频率

In [ ]:
from QubitEnergyFineSimulation import XmonEnergy
Cd = 76.9e-15+9.8e-15
R=13000
ng = 0
elementParameter = [Cd,R,ng]
Xmon1 = XmonEnergy(elementParameter)
el0 = Xmon1.energyLevel
print(el0[1])
print(el0[2]-2*el0[1])
ng = 0.5
elementParameter = [Cd,R,ng]
Xmon1 = XmonEnergy(elementParameter)
el1 = Xmon1.energyLevel
print(el1[1])
print(el1[2]-2*el1[1])
print(el1[1]-el0[1])
print((el1[2]-el1[1])-(el0[2]-el0[1]))

In [ ]:
from QubitEnergyFineSimulation import XmonEnergy
from scipy.optimize import curve_fit
Cd = 76.9e-15+9.8e-15
Rlist = np.linspace(8000,14000,61)
FreqList = []
Time = []
for R in Rlist:
    ng = 0
    elementParameter = [Cd,R,ng]
    Xmon1 = XmonEnergy(elementParameter)
    el0 = Xmon1.energyLevel
    
    ng = 0.5
    elementParameter = [Cd,R,ng]
    Xmon1 = XmonEnergy(elementParameter)
    el1 = Xmon1.energyLevel
    FreqList.append(el1[1])
    delta = (el1[2]-el1[1])-(el0[2]-el0[1])
    Time.append(1/2/delta)     
    
def func(x, a, b, c):
    return a * np.exp(-b * x) + c
popt, pcov = curve_fit(func, FreqList, Time)
yy = [func(x,popt[0],popt[1],popt[2]) for x in FreqList]
plt.figure()
plt.plot(FreqList,Time,linewidth = 2,label='Simulation');plt.plot(FreqList,yy,linewidth = 2,label = 'Time='+str(round(popt[0],6))+'*exp('+str(-round(popt[1],2))+'*x)'+'+'+str(round(popt[2],2)))
plt.xlabel('Qubit Frequency (GHz)',fontsize=15)
plt.ylabel('Integral Time (ns)',fontsize=15)
plt.yscale('log')
ax = plt.gca();ax.spines['bottom'].set_linewidth(1);ax.spines['left'].set_linewidth(1)
plt.xticks(size = 15);plt.yticks(size = 15)
plt.legend()


## 由实验测得的非谐性和顶点频率拟合电容和电阻

In [ ]:

import scipy.optimize
from QubitEnergyFineSimulation import XmonEnergy
def XmonParameter(Cd,R):
    Cd = Cd+9.8e-15
    ng = 0
    elementParameter = [Cd,R,ng]
    Xmon1 = XmonEnergy(elementParameter)
    energyLevel = Xmon1.energyLevel
    return([energyLevel[1],energyLevel[2]-2*energyLevel[1]])
    
def CapAndResGet(f01Max,harmonicity):
    def cosfunction(para):
        qubitPara = XmonParameter(para[0],para[1])
        # print(energyLevel)
        cost = abs(qubitPara[0]-f01Max)+abs(qubitPara[1]-harmonicity)
        return(cost)
    beta0 = [82e-15,10000]
    res=scipy.optimize.minimize(cosfunction,beta0,method='Nelder-Mead')
    # print(XmonParameter(res.x[0],res.x[1]))
    return(res.x)
f01List = [5.08 ]
harmonicityList = [-0.252]
CdList = []
RList= []
f01optList = []
anaoptList = []
for ii in range(len(f01List)):
    optPara = CapAndResGet(f01List[ii],harmonicityList[ii])
    optQubit = XmonParameter(optPara[0],optPara[1])
    CdList.append(optPara[0]);RList.append(optPara[1])
    f01optList.append(optQubit[0]);anaoptList.append(optQubit[1])
print(CdList)
print(RList)
print(f01optList)
print(anaoptList)

## 计算控制线损耗和驱动强度

In [ ]:
## 统一驱动电压
e = 1.602e-19
phi0 = 2.067833e-15
vDrive = 0.006
attenuator = -41
vDriveMC = vDrive*10**(attenuator/20)
RLine = 50
iDriveMC = vDriveMC/RLine


### 电容驱动

In [ ]:
from QubitEnergyFineSimulation import XmonEnergy
Cd = 76.74e-15+9.8e-15
Cdrive = 36.5e-18
R=10000
ng = 0
elementParameter = [Cd,R,ng]
Xmon1 = XmonEnergy(elementParameter)
el = Xmon1.energyLevel
omega = (el[1]-el[0])*2*np.pi*1e9

## 电容驱动的损耗
T1_C = Cd/(omega**2*Cdrive**2*RLine)

## 电容驱动的强度
gc = Cdrive*vDriveMC/e*(8*Xmon1.Ej*Xmon1.Ec**3)**(1/4)/2/np.pi ## 单位为GHz

print('比特频率为:{0} GHz,\n电容驱动等效T1为:{1} us,\n电容驱动等效强度为:{2} MHz'.format(round(omega/1e9/2/np.pi,4),round(T1_C*1e6,1),round(gc*1e3,1))
      )


### 电感驱动

In [ ]:
from QubitEnergyFineSimulation import XmonEnergy
Cd = 76.74e-15+9.8e-15
mInductance = 1e-12
R=10000
ng = 0
elementParameter = [Cd,R,ng]
Xmon1 = XmonEnergy(elementParameter)
el = Xmon1.energyLevel
omega = (el[1]-el[0])*2*np.pi*1e9

## 电感驱动的损耗
T1_M = RLine/(omega**4*mInductance**2*Cd)
## 电容驱动的强度

gm = 2*np.pi*mInductance*iDriveMC/phi0*(2*Xmon1.Ej**3*Xmon1.Ec)**(1/4)/2/np.pi ## 单位为GHz

print('比特频率为:{0} GHz,\n电感驱动等效T1为:{1} us,\n电感驱动等效强度为:{2} MHz'.format(round(omega/1e9/2/np.pi,4),round(T1_M*1e6,1),round(gm*1e3,1))
      )

## 接地比特直接耦合 

### 计算接地比特间耦合强度

In [ ]:
from QubitSimulation import Xmon
Cd = 76.80e-15+9.8e-15
Cc = 0.1e-15
R=9000

Capa = np.array([
    [Cd,Cc],
    [Cc,Cd]
])
Linv = np.ones_like(Capa)*1e9
RNAN = 1e9
RList = np.array([
    [R,RNAN],
    [RNAN,R]
])
flux = np.zeros_like(Capa)
Nlevel = [20,20]
para = [Capa,Linv,RList,flux,Nlevel]
DT = Xmon(para)
Hamilton = DT.GetHamilton()
[energyEig,stateEig] = Hamilton.eigenstates()
energyLevel = (energyEig-energyEig[0])/2/np.pi
print((energyEig[2]-energyEig[1])/2/np.pi/2)

### 接地比特与读取腔的耦合强度

In [ ]:
from QubitEnergyFineSimulation import XmonEnergy
Cc = 4.4e-15
Cq = 86.3e-15+9.8e-15-Cc
CqAll = Cq+Cc
R=9000
wr = 6.2e9


Cr = np.pi/4/50/2/np.pi/wr
ng = 0
elementParameter = [CqAll,R,ng]
Xmon1 = XmonEnergy(elementParameter)
el = Xmon1.energyLevel
wq = el[1]*1e9

Cqq = (Cq*Cr+Cq*Cc+Cr*Cc)/(Cr+Cc)
Crr = (Cq*Cr+Cq*Cc+Cr*Cc)/(Cq+Cc)
Ccc = (Cq*Cr+Cq*Cc+Cr*Cc)/(Cc)

g = np.sqrt(wq*wr)*np.sqrt(Cqq*Crr)/2/Ccc
print(wq/1e9)
print(g/1e6)

## 接地比特-接地coupler-接地比特 1V4 的系统能级

In [ ]:
from QubitEnergyFineSimulation import XmonEnergy1V4

CJ=9.8e-15;     SRatio=5.0; 

C12 = 20e-15
C23 = C12
C13 = 0.0e-15

C1=90.5e-15-C12*4-C13*4+CJ
C2=103e-15-C12-C23+CJ*SRatio
C3=90.5e-15-C23*4-C13*4+CJ

# C1=83e-15
# C2=97e-15
# C3=83e-15

R1=7000
R2 = R1/SRatio

Capa = np.array([
    [C1,    C12,    C13,    C12,  C12,    C12,    0,    0,    0],
    [C12,   C2,     C23,    0,    0,      0,      0,    0,    0],
    [C13,   C23,    C3,     0,    0,      0,      C23,  C23,  C23],
    [C12,   0,      0,      C2,   0,      0,      0,    0,    0],
    [C12,   0,      0,      0,    C2,     0,      0,    0,    0],
    [C12,   0,      0,      0,    0,      C2,     0,    0,    0],
    [0,     0,      C23,    0,    0,      0,      C2,   0,    0],
    [0,     0,      C23,    0,    0,      0,      0,    C2,   0],
    [0,     0,      C23,    0,    0,      0,      0,    0,    C2],

])
Rlist = [R1,R2,R1,R2,R2,R2,R2,R2,R2]
para = [Capa,Rlist]
FTWC = XmonEnergy1V4(para)
el = FTWC.energyLevel

RlistWithoutC = [R1,10,R1,10,10,10,10,10,10]
paraWithoutC = [Capa,RlistWithoutC]
FTWithoutC = XmonEnergy1V4(paraWithoutC)
elWithoutC = FTWithoutC.energyLevel


couplerFrequency = el[FTWC.stateIndexList[3]]
qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
couplingXYQQDirect = -abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2 if FTWithoutC.couplingMinus else abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2
anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
deltaFreqency = (qubitFreuqency-couplerFrequency)
couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
CouplerLeakage = FTWC.couplerLeakage
print(el[1:8])
print(elWithoutC[1:8])
print('Qubit Frequency:'+str(qubitFreuqency))
print('Coupler Frequency:'+str(couplerFrequency))
print('Anharmonicity:'+str(anharmonicity))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQ))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQ))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirect))
print('Qubit-Coupler couling:'+str(couplingQC))
print('CouplerLeakage:'+str(CouplerLeakage))


### 系统能级随coupler频率变化关系


In [ ]:
%%writefile GQGC.py
def func(flux,Capa,Rlist):
    import numpy as np
    from QubitEnergyFineSimulation import XmonEnergy1V4
    RlistNew = Rlist.copy()
    RCoupler = Rlist[1]/np.cos(np.pi*flux)
    RlistNew[1] = RCoupler
    para = [Capa,RlistNew]
    FTWC = XmonEnergy1V4(para)
    el = FTWC.energyLevel
    qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
    qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
    coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
    anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
    coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
    result = {
        'el':el[0:8],
        'coulingXYQCQ':coulingXYQCQ,
        'coulingZZQCQ':coulingZZQCQ,
        'anharmonicity':anharmonicity,
        'couplerLeakage':FTWC.couplerLeakage,
    }
    return(result)

In [ ]:
import matplotlib.pyplot as plt
import GQGC
%matplotlib
fluxList = np.linspace(0.2,0.45,101)
freqList = []
couplingXYList = []
couplingZZList = []
anharmonicityList = []
CouplerLeakageList = []


numCores = int(mp.cpu_count())
pool = mp.Pool(numCores)
resultList = [pool.apply_async(GQGC.func, args=(flux,Capa,Rlist,)) for flux in fluxList]
pool.close()
pool.join()
resultList = [p.get() for p in resultList]
    
freqList = [result['el'] for result in resultList]
couplingXYList = [result['coulingXYQCQ'] for result in resultList]
couplingZZList = [result['coulingZZQCQ'] for result in resultList]
anharmonicityList = [result['anharmonicity'] for result in resultList]
CouplerLeakageList = [result['couplerLeakage'] for result in resultList]
  
freqList = np.transpose(freqList)



In [ ]:
plt.figure(1,dpi = 200)

for ii in range(len(freqList)):
    plt.plot(fluxList,freqList[ii],label = str(ii))
plt.xlabel('flux');plt.ylabel('Frequency (GHz)');plt.legend()
plt.figure(2,dpi = 200)
plt.plot(fluxList,couplingXYList,label = 'XY')
plt.plot(fluxList,couplingZZList,label = 'ZZ')
plt.xlabel('flux');plt.ylabel('Coupling Strength (GHz)')
plt.legend()

plt.figure(3,dpi = 200)
plt.plot(fluxList,anharmonicityList)
plt.xlabel('flux');plt.ylabel('Anharmonicity (GHz)')
plt.tight_layout()
plt.show()

plt.figure(4,dpi = 200)
plt.plot(fluxList,CouplerLeakageList)
plt.xlabel('flux');plt.ylabel('CouplerLeakageList')
plt.tight_layout()
plt.show()

### 关断点和10M耦合情况


In [ ]:
fluxRange = np.argmin(couplingXYList)
## coupling off
offIndex = np.argmin(np.abs(couplingXYList[0:fluxRange]))
offFlux = fluxList[offIndex]
offFreq = couplerFreqList[offIndex]
print([offFlux,offFreq,CouplerLeakageList[offIndex]*100])

## 耦合g

CZg = 0.016
coupling10Index = np.argmin(np.abs(np.array(couplingXYList[0:fluxRange])+CZg))
coupling10Flux = fluxList[coupling10Index]
coupling10Freq = couplerFreqList[coupling10Index]
print([CZg,coupling10Flux,coupling10Freq,CouplerLeakageList[coupling10Index]*100])

### coupler导致的比特dephasing


In [ ]:
phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (freqList[1][phiIndex-1]-freqList[1][phiIndex+1])/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = freqList[1][0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000)

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (freqList[1][phiIndex-1]-freqList[1][phiIndex+1])/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = freqList[1][0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000)

### Coupler热激发导致的dephasing


In [ ]:
phiSet = offFlux
T1 = 20e-6
phiIndex = np.argmin(abs(fluxList-phiSet))
wq = freqList[1][phiIndex]*1e9
wc = freqList[3][phiIndex]*1e9

chi = -(couplingQC*1e9)**2/(wq-wc)*1/(1+(wq-wc)/(anharmonicityList[phiIndex]*1e9))

k = 1.380649e-23;h = 6.62607015e-34;
nc = np.exp(-h*wc/k/50e-3)/(1+np.exp(-h*wc/k/50e-3))
kappa = 1/T1

gamma = nc*kappa*chi**2/(kappa**2/4+chi**2)
T2 = 1/gamma
print(T2/1e-6)


phiSet = coupling10Flux
T1 = 20e-6
phiIndex = np.argmin(abs(fluxList-phiSet))
wq = freqList[1][phiIndex]*1e9
wc = freqList[3][phiIndex]*1e9

chi = -(couplingQC*1e9)**2/(wq-wc)*1/(1+(wq-wc)/(anharmonicityList[phiIndex]*1e9))

k = 1.380649e-23;h = 6.62607015e-34;
nc = np.exp(-h*wc/k/50e-3)/(1+np.exp(-h*wc/k/50e-3))
kappa = 1/T1

gamma = nc*kappa*chi**2/(kappa**2/4+chi**2)
T2 = 1/gamma
print(T2/1e-6)

## 接地比特-浮地coupler-接地比特 1V4 的系统能级

In [101]:
from QubitEnergyFineSimulation import GroundedTransmonWithFloatingCoupler1V4
CJ = 9.8e-15;SRatio = 7


C12=6.825e-15
# C12=8.535e-15
C13=0.08e-15
C23Raw=26.83e-15; C23 = C23Raw+CJ*SRatio
# C23Raw=46.83e-15; C23 = C23Raw+CJ*SRatio
C24=C12
C34=C13
    
C1=77.428e-15-4*C12-4*C13+CJ
C2=213.581e-15-C23Raw-C12-C24
C3=164.728e-15-C23Raw-C13-C34

# C2=293.581e-15-C23Raw-C12-C24
# C3=244.728e-15-C23Raw-C13-C34
C4=C1

R=11000

Capa = np.array([
    [C1,    C12,    C13,    0,      C12,      C13,      C12,    C13,    C12,    C13,    0,     0,    0,      0,      0,      0,   ],
    [C12,   C2,     C23,    C24,    0,        0,        0,      0,      0,      0,      0,     0,    0,      0,      0,      0,   ],
    [C13,   C23,    C3,     C34,    0,        0,        0,      0,      0,      0,      0,     0,    0,      0,      0,      0,   ],
    [0,     C24,    C34,    C4,     0,        0,        0,      0,      0,      0,      C24,   C34,  C24,    C34,    C24,    C34, ],
    [C12,   0,      0,      0,      C2,       C23,      0,      0,      0,      0,      0,     0,    0,      0,      0,      0,   ],
    [C13,   0,      0,      0,      C23,      C3,       0,      0,      0,      0,      0,     0,    0,      0,      0,      0,   ],
    [C12,   0,      0,      0,      0,        0,        C2,     C23,    0,      0,      0,     0,    0,      0,      0,      0,   ],
    [C13,   0,      0,      0,      0,        0,        C23,    C3,     0,      0,      0,     0,    0,      0,      0,      0,   ],
    [C12,   0,      0,      0,      0,        0,        0,      0,      C2,     C23,    0,     0,    0,      0,      0,      0,   ],
    [C13,   0,      0,      0,      0,        0,        0,      0,      C23,    C3,     0,     0,    0,      0,      0,      0,   ],
    [0,     0,      0,      C24,    0,        0,        0,      0,      0,      0,      C2,    C23,  0,      0,      0,      0,   ],
    [0,     0,      0,      C34,    0,        0,        0,      0,      0,      0,      C23,   C3,   0,      0,      0,      0,   ],
    [0,     0,      0,      C24,    0,        0,        0,      0,      0,      0,      0,     0,    C2,     C23,    0,      0,   ],
    [0,     0,      0,      C34,    0,        0,        0,      0,      0,      0,      0,     0,    C23,    C3,     0,      0,   ],
    [0,     0,      0,      C24,    0,        0,        0,      0,      0,      0,      0,     0,    0,      0,      C2,     C23, ],
    [0,     0,      0,      C34,    0,        0,        0,      0,      0,      0,      0,     0,    0,      0,      C23,    C3,  ],
])
Rlist = [R,R/SRatio,R,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio]
para = [Capa,Rlist]
FTWC = GroundedTransmonWithFloatingCoupler1V4(para)
el = FTWC.energyLevel
RlistWithoutC = [R,50,R,50,50,50,50,50,50]
paraWithoutC = [Capa,RlistWithoutC]
FTWithoutC = GroundedTransmonWithFloatingCoupler1V4(paraWithoutC)
elWithoutC = FTWithoutC.energyLevel


couplerFrequency = el[FTWC.stateIndexList[3]]
qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
couplingXYQQDirect = -abs(elWithoutC[2]-elWithoutC[1])/2 if FTWithoutC.couplingMinus else abs(elWithoutC[2]-elWithoutC[1])/2
anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
deltaFreqency = (qubitFreuqency-couplerFrequency)
couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
# couplerAnharmonicity = el[FTWC.stateIndexList[7]]-2*couplerFrequency
CouplerLeakage = FTWC.couplerLeakage

print(el[1:8])
print(elWithoutC[1:8])
print('Qubit Frequency:'+str(qubitFreuqency))
print('Coupler Frequency:'+str(couplerFrequency))
print('Anharmonicity:'+str(anharmonicity))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQ))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQ))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirect))
print('Qubit-Coupler couling:'+str(couplingQC))
print('CouplerLeakage:'+str(CouplerLeakage))
# print('Coupler Anharmonicity:'+str(couplerAnharmonicity))

[ 4.51967119  4.52645743  8.75462186  8.79332992  8.79350568  9.04630426
 12.80495752]
[ 4.51969785  4.52735589  8.79404417  8.79426344  9.04727264 12.80585038
 12.80586104]
Qubit Frequency:4.523064311102017
Coupler Frequency:8.754621860915382
Anharmonicity:-0.25271082234273834
Qubit-Coupler-Qubit XY couling:0.003393121244899877
Qubit-Coupler-Qubit ZZ couling:0.00017564116788904016
Qubit-Qubit direct couling:0.0038290217426029116
Qubit-Coupler couling:0.04294808542906982
CouplerLeakage:0.0006077714991201383


### 电容和电感引入的损耗

In [ ]:
## Capacitive loss
e = 1.60217663410e-19
h = 6.62607015e-34;hbar = h/2/np.pi
kb = 1.380649e-23;T = 0.01
fluxList = np.linspace(0.0,0.3,21)
freqList = []

T1CList = []
for flux in fluxList:
    RR = R/np.cos(np.pi*flux)
    RSweepList = [RR,R/SRatio,R,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio]
    paraSweep = [Capa,RSweepList]
    FTWCSweep = GroundedTransmonWithFloatingCoupler1V4(paraSweep)
    operator  = FTWCSweep.DT.nn[0]
    state0 = FTWCSweep.stateEig[0];state1 = FTWCSweep.stateEig[FTWCSweep.stateIndexList[2]]
    freq = FTWCSweep.energyLevel[FTWCSweep.stateIndexList[2]]*2*np.pi*1e9
    gammaC = 1/(hbar**2)*(2*e*abs((state0.dag()*operator*state1).data.toarray()[0,0]))**2*hbar*freq*(1+1/np.tanh(hbar*freq/2/kb/T))*1/(freq*1e6*(2*np.pi*6e9/freq)**0.7*C1)


    freqList.append(freq/2/np.pi/1e9)
    T1CList.append(1/gammaC)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib

plt.figure(1,dpi = 200)

plt.plot(freqList,T1CList)
plt.xlabel('Frequency(GHz)');plt.ylabel('T1CList (us)')
plt.legend()

In [ ]:
e = 1.60217663410e-19
h = 6.62607015e-34;hbar = h/2/np.pi
flux = 0
RR = R/np.cos(np.pi*flux)
RSweepList = [RR,RR/SRatio,RR,RR/SRatio,RR/SRatio,RR/SRatio,RR/SRatio,RR/SRatio,RR/SRatio]
paraSweep = [Capa,RSweepList]
FTWCSweep = GroundedTransmonWithFloatingCoupler1V4(paraSweep)
operator  = 2*e*FTWCSweep.DT.nn[0]
state0 = FTWCSweep.stateEig[0];state1 = FTWCSweep.stateEig[FTWCSweep.stateIndexList[2]]
gamma = 1/hbar**2*abs((state0.dag()*operator*state1).data.toarray()[0,0])**2*hbar*


### 系统能级随coupler频率变化关系


In [102]:
%%writefile GQFC.py
def func(flux,Capa,Rlist,couplingXYQQDirect):
    import numpy as np
    from QubitEnergyFineSimulation import GroundedTransmonWithFloatingCoupler1V4 
    RlistNew = Rlist.copy()
    RCoupler = Rlist[1]/np.cos(np.pi*flux)
    RlistNew[1] = RCoupler
    para = [Capa,RlistNew]
    FTWC = GroundedTransmonWithFloatingCoupler1V4(para)
    el = FTWC.energyLevel
    couplerFrequency = el[FTWC.stateIndexList[3]]
    qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
    qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
    coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
    anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
    coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
    deltaFreqency = (qubitFreuqency-couplerFrequency)
    couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
    result = {
        'couplerFrequency':couplerFrequency,
        'qubitFreuqency':qubitFreuqency,
        'el':el[0:8],
        'coulingXYQCQ':coulingXYQCQ,
        'coulingZZQCQ':coulingZZQCQ,
        'anharmonicity':anharmonicity,
        'couplingQC':couplingQC,
        'couplerLeakage':FTWC.couplerLeakage,
    }
    return(result)

Overwriting GQFC.py


In [103]:
import matplotlib.pyplot as plt
import GQFC
%matplotlib
fluxList = np.linspace(0.10,0.44,201)
freqList = []
couplingXYList = []
couplingZZList = []
anharmonicityList = []
CouplerLeakageList = []
   

numCores = int(mp.cpu_count())
pool = mp.Pool(numCores)
resultList = [pool.apply_async(GQFC.func, args=(flux,Capa,Rlist,couplingXYQQDirect)) for flux in fluxList]
pool.close()
pool.join()
resultList = [p.get() for p in resultList]

qubitFreqList = [result['qubitFreuqency'] for result in resultList]
couplerFreqList = [result['couplerFrequency'] for result in resultList]
freqList = [result['el'] for result in resultList]
couplingXYList = [result['coulingXYQCQ'] for result in resultList]
couplingZZList = [result['coulingZZQCQ'] for result in resultList]
anharmonicityList = [result['anharmonicity'] for result in resultList]
couplingQCList = [result['couplingQC'] for result in resultList]
CouplerLeakageList = [result['couplerLeakage'] for result in resultList]
  
freqList = np.transpose(freqList)



Using matplotlib backend: QtAgg


In [35]:
%matplotlib
plt.figure(1,dpi = 200)

for ii in range(len(freqList)):  
    plt.plot(fluxList,freqList[ii],label = str(ii))
plt.xlabel('flux');plt.ylabel('Frequency (GHz)');plt.legend()

plt.figure(2,dpi = 200)
plt.plot(fluxList,couplingXYList,label = 'XY')
plt.plot(fluxList,couplingZZList,label = 'ZZ')
plt.xlabel('flux');plt.ylabel('Coupling Strength (GHz)')
plt.legend()
 
plt.figure(3,dpi = 200)
plt.plot(fluxList,anharmonicityList)
plt.xlabel('flux');plt.ylabel('Anharmonicity (GHz)')
plt.tight_layout()
plt.show()

plt.figure(4,dpi = 200)
plt.plot(fluxList,CouplerLeakageList)
plt.xlabel('flux');plt.ylabel('CouplerLeakageList')
plt.tight_layout()
plt.show()

plt.figure(5,dpi = 200)
plt.plot(fluxList,couplingQCList)
plt.xlabel('flux');plt.ylabel('QC Coupling')
plt.tight_layout()
plt.show()

### 关断点和10M耦合情况


In [104]:
fluxRange = np.argmin(couplingXYList)
## coupling off
offIndex = np.argmin(np.abs(couplingXYList[0:fluxRange]))
offFlux = fluxList[offIndex]
offFreq = couplerFreqList[offIndex]
print([offFlux,offFreq,CouplerLeakageList[offIndex]*100])

## 耦合g

CZg = 0.016
coupling10Index = np.argmin(np.abs(np.array(couplingXYList[0:fluxRange])+CZg))
coupling10Flux = fluxList[coupling10Index]
coupling10Freq = couplerFreqList[coupling10Index]
print([CZg,coupling10Flux,coupling10Freq,CouplerLeakageList[coupling10Index]*100])


[0.3839, 5.192234449781756, 1.4233177245401085]
[0.016, 0.40769999999999995, 4.668991508996792, 22.430504361584]


### coupler导致的比特dephasing


In [ ]:
phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (freqList[1][phiIndex-1]+freqList[2][phiIndex-1]-freqList[1][phiIndex+1]-freqList[2][phiIndex+1])/2/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = freqList[1][0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000)

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (freqList[1][phiIndex-1]+freqList[2][phiIndex-1]-freqList[1][phiIndex+1]-freqList[2][phiIndex+1])/2/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = freqList[1][0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000)

### Coupler T2导致的比特T2


#### 关断点

In [105]:
### 定义噪声强度和Coupler磁通偏置点
phi0 = 2.067833e-15;h = 6.62607015e-34;hbar = h/2/np.pi
AphiCouplerWhite = (1.05e-8*phi0)**2
AphiCouplerF = (46e-6*phi0)


phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
freqCoupler = couplerFreqList[phiIndex]

delta = qubitFreqList[phiIndex]-couplerFreqList[phiIndex]
gCoupling = couplingQCList[phiIndex]

### 计算coupler Tphi1和 Tphi2
slope = (couplerFreqList[phiIndex-1]-couplerFreqList[phiIndex+1])*2*np.pi*1e9/(fluxList[phiIndex-1]-fluxList[phiIndex+1])/phi0
gammaPhi1Coupler = AphiCouplerWhite*abs(slope)**2
gammaPhi2Coupler = AphiCouplerF*abs(slope)

TPhi1Coupler = 4/gammaPhi1Coupler*1e6
TPhi2Coupler = 1/gammaPhi2Coupler*1e6
print('Coupler在关断点，coupler磁通为{phi}Phi0的coupler Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Coupler,4),data2 = round(TPhi2Coupler,4)))
### 对比特T2影响
leakage = CouplerLeakageList[phiIndex]
TPhi1Qubit = TPhi1Coupler/leakage
TPhi2Qubit = TPhi2Coupler/leakage
print('Coupler在关断点，coupler磁通为{phi}Phi0的qubit Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Qubit,4),data2 = round(TPhi2Qubit,4)))

Coupler在关断点，coupler磁通为0.384Phi0的coupler Tphi1为1.9873 us, Tphi2为0.1609 us:
Coupler在关断点，coupler磁通为0.384Phi0的qubit Tphi1为139.6264 us, Tphi2为11.3041 us:


#### 做门点（10M）

In [106]:
### 定义噪声强度和Coupler磁通偏置点
phi0 = 2.067833e-15;h = 6.62607015e-34;hbar = h/2/np.pi
AphiCouplerWhite = (1.05e-8*phi0)**2
AphiCouplerF = (46e-6*phi0)

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
freqCoupler = couplerFreqList[phiIndex]


delta = qubitFreqList[phiIndex]-couplerFreqList[phiIndex]
gCoupling = couplingQCList[phiIndex]
### 计算coupler T2
slope = (couplerFreqList[phiIndex-1]-couplerFreqList[phiIndex+1])*2*np.pi*1e9/(fluxList[phiIndex-1]-fluxList[phiIndex+1])/phi0

gammaPhi1Coupler = AphiCouplerWhite*abs(slope)**2
gammaPhi2Coupler = AphiCouplerF*abs(slope)

TPhi1Coupler = 4/gammaPhi1Coupler*1e6
TPhi2Coupler = 1/gammaPhi2Coupler*1e6
print('Coupler在CZ做门点，coupler磁通为{phi}Phi0的coupler Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Coupler,4),data2 = round(TPhi2Coupler,4)))
### 对比特T2影响
leakage = CouplerLeakageList[phiIndex]
TPhi1Qubit = TPhi1Coupler/leakage
TPhi2Qubit = TPhi2Coupler/leakage
print('Coupler在CZ做门点，coupler磁通为{phi}Phi0的qubit Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Qubit,4),data2 = round(TPhi2Qubit,4)))

### 做门错误
tgate = 1/2/np.sqrt(2)/0.8/CZg
errorPhi1 = 2*1/2*(tgate/(TPhi1Qubit*1000))
errorPhi2 = 2*1/2*(tgate/(TPhi2Qubit*1000))**2

errorCZ = errorPhi1+errorPhi2
print('Coupler在CZ做门点，做门时间为{time}ns，coupler引入的错误为：{data}%,其中白噪声引入的错误为：{error1}%，1/f噪声引入的错误为：{error2}%:'.format(time = round(tgate,1),data = round(errorCZ*100,4),error1 = round(errorPhi1*100,4),error2 = round(errorPhi2*100,4)))

Coupler在CZ做门点，coupler磁通为0.408Phi0的coupler Tphi1为2.4478 us, Tphi2为0.1786 us:
Coupler在CZ做门点，coupler磁通为0.408Phi0的qubit Tphi1为10.9129 us, Tphi2为0.7961 us:
Coupler在CZ做门点，做门时间为27.6ns，coupler引入的错误为：0.3735%,其中白噪声引入的错误为：0.2531%，1/f噪声引入的错误为：0.1204%:


### Coupler热激发导致的dephasing


In [107]:
def DephasingCouplerThermal(phiSet):
    T1Ccoupler = 5e-6
    T_est = 60e-3
    phiIndex = np.argmin(abs(fluxList-phiSet))

    wc = freqList[3][phiIndex]*2*np.pi*1e9
    wq = (freqList[1][phiIndex]+freqList[2][phiIndex])/2*2*np.pi*1e9
    hbar=1.054560652926899e-34;     
    h = hbar*2*np.pi;     
    e = 1.60217662e-19;     
    phi0 = h/2/e
    kb = 1.380649e-23
    R = 50
    
    
    nc = 1/(np.exp(hbar*wc/kb/T_est)-1)
    chi = -(couplingQCList[phiIndex]*1e9*2*np.pi)**2/(wq-wc)*1/(1+(wq-wc)/(anharmonicityList[phiIndex]*1e9*2*np.pi))
    kb = 1.380649e-23;h = 6.62607015e-34;hbar = h/2/np.pi
    kappa = 1/T1Ccoupler

    eta = kappa**2/(kappa**2+4*chi**2)
    gammaPhi = eta*4*chi**2/kappa*nc
    Tphi = 1/gammaPhi
    errorPhi1 = 2*1/2*(tgate/1e9/(Tphi))
    print(nc,tgate,Tphi/1e-6,errorPhi1*100)




phiSet = offFlux
DephasingCouplerThermal(phiSet)
phiSet = coupling10Flux
DephasingCouplerThermal(phiSet)







0.015966732139406702 27.62135864009951 313.22332805685284 0.008818423203486933
0.024467948262630313 27.62135864009951 204.34938240349368 0.013516732135534596


## 接地比特-接地coupler-接地比特 1V4 的系统能级（包含耦合电容）

In [ ]:
from QubitEnergyFineSimulation import GroundedTransmonWithGroundCoupler
CJ=9.8e-15;     SRatio=6.0; 

C12=3e-15
C13=0.00e-15
C14=3e-15
C24 = 10e-15

C23=C12
C34=C14
    
C1=76.428e-15-4*C12-4*C13-4*C14+CJ
C2=200e-15-C12-C23-C24
C3=C1
C4=70e-15-C14-C24-C34

R=10000

Capa = np.array([  
    [ C1,   C12,    C13,    C14,      C12,      C13,    C14,      C12,      C13,    C14,      C12,      C13,    C14,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C12,    C2,    C23,    C24,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C13,   C23,     C3,    C34,        0,        0,      0,        0,        0,      0,        0,        0,      0,      C13,      C23,    C34,      C13,      C23,    C34,      C13,      C23,    C34],
    [C14,   C24,    C34,     C4,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C12,     0,      0,     0 ,       C2,      C23,    C24,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C13,     0,      0,     0 ,      C23,       C3,    C34,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C14,     0,      0,     0 ,      C24,      C34,     C4,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C12,     0,      0,     0 ,        0,        0,      0,       C2,      C23,    C24,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C13,     0,      0,     0 ,        0,        0,      0,      C23,       C3,    C34,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C14,     0,      0,     0 ,        0,        0,      0,      C24,      C34,     C4,        0,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C12,     0,      0,     0 ,        0,        0,      0,       0 ,        0,      0,       C2,      C23,    C24,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C13,     0,      0,     0 ,        0,        0,      0,       0 ,        0,      0,      C23,       C3,    C34,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [C14,     0,      0,     0 ,        0,        0,      0,       0 ,        0,      0,      C24,      C34,     C4,        0,        0,      0,        0,        0,      0,        0,        0,      0],
    [0  ,     0,    C13,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,       C1,      C12,    C14,        0,        0,      0,        0,        0,      0],
    [0  ,     0,    C23,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,      C12,       C2,    C24,        0,        0,      0,        0,        0,      0],
    [0  ,     0,    C34,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,      C14,      C24,     C4,        0,        0,      0,        0,        0,      0],
    [0  ,     0,    C13,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,        0,        0,      0,       C1,      C12,    C14,        0,        0,      0],
    [0  ,     0,    C23,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,        0,        0,      0,      C12,       C2,    C24,        0,        0,      0],
    [0  ,     0,    C34,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,        0,        0,      0,      C14,      C24,     C4,        0,        0,      0],
    [0  ,     0,    C13,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,       C1,      C12,    C14],
    [0  ,     0,    C23,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,      C12,       C2,    C24],
    [0  ,     0,    C34,     0 ,        0,        0,      0,       0 ,        0,      0,        0,        0,      0,        0,        0,      0,        0,        0,      0,      C14,      C24,     C4],




])  
Rlist = [R,R/SRatio,R]
para = [Capa,Rlist]
FTWC = GroundedTransmonWithGroundCoupler(para)
el = FTWC.energyLevel
RlistWithoutC = [R,50,R]
paraWithoutC = [Capa,RlistWithoutC]
FTWithoutC = GroundedTransmonWithGroundCoupler(paraWithoutC)
elWithoutC = FTWithoutC.energyLevel


couplerFrequency = el[FTWC.stateIndexList[3]]
qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
couplingXYQQDirect = -abs(elWithoutC[2]-elWithoutC[1])/2 if FTWithoutC.couplingMinus else abs(elWithoutC[2]-elWithoutC[1])/2
anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
deltaFreqency = (qubitFreuqency-couplerFrequency)
couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
CouplerLeakage = FTWC.couplerLeakage

print(el[1:8])
print(elWithoutC[1:8])
print('Qubit Frequency:'+str(qubitFreuqency))
print('Coupler Frequency:'+str(couplerFrequency))
print('Anharmonicity:'+str(anharmonicity))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQ))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQ))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirect))
print('Qubit-Coupler couling:'+str(couplingQC))
print('CouplerLeakage:'+str(CouplerLeakage))

# 浮地比特

## 单独的浮地比特

In [ ]:
from QubitEnergyFineSimulation import SingleFloatingTransmon

C12=3.89e-15   
C1=157.78e-15-C12
C2=158.95e-15-C12
R=8500
CJ=9.8e-15; C12 =C12+CJ 

# C12=185.7e-15/2/np.pi   
    
# C1=853.27e-15/2/np.pi-C12
# C2=802.11e-15/2/np.pi-C12   
# R=9000
# CJ=9.8e-15;  

Capa = np.array([
    [C1,C12],
    [C12,C2],
    
])

elementParameter = [Capa,R,0]
FT = SingleFloatingTransmon(elementParameter)
el = FT.energyLevel
print(el[1]-el[0])
print(el[2]-el[0]-2*el[1])
print(el[0:10])

## 由实验测得的非简谐性和顶点频率拟合电容和电阻

In [ ]:
import scipy.optimize
from QubitEnergyFineSimulation import SingleFloatingTransmon
def DFTParameter(CRatio,R):
    C12=29.68e-15*CRatio  
    
    C1=136.4e-15*CRatio-C12
    C2=128.2e-15*CRatio-C12   
    CJ=9.8e-15; 
    C12 =C12+CJ
    Capa = np.array([
        [C1,C12],
        [C12,C2],
        
    ])
    elementParameter = [Capa,R,0]
    FT = SingleFloatingTransmon(elementParameter)
    energyLevel = FT.energyLevel
    return([energyLevel[1],energyLevel[2]-2*energyLevel[1],C1,C2,C12-CJ])
    
def CapAndResGet(f01Max,harmonicity):
    def costfunction(para):
        qubitPara = DFTParameter(para[0],para[1])
        # print(energyLevel)
        cost = abs(qubitPara[0]-f01Max)+abs(qubitPara[1]-harmonicity)
        return(cost)
    
    beta0 = [1,10000]
    res=scipy.optimize.minimize(costfunction,beta0,method='Nelder-Mead')
    # print(DFTParameter(res.x[0],res.x[1]))
    return(res.x)

f01List = [4.220,4.242,4.327]
harmonicityList = [-0.260,-0.256,-0.256]
RatioList = []
C1List = []
C2List = []
C12List = []
RList= []
f01optList = []
anaoptList = []
for ii in range(len(f01List)):
    optPara = CapAndResGet(f01List[ii],harmonicityList[ii])
    optQubit = DFTParameter(optPara[0],optPara[1])
    RatioList.append(optPara[0]);RList.append(optPara[1])
    f01optList.append(optQubit[0]);anaoptList.append(optQubit[1]);C1List.append(optQubit[2]);C2List.append(optQubit[3]);C12List.append(optQubit[4]);
print(RatioList)
print(RList)
print(f01optList)
print(anaoptList)
print(C1List)
print(C2List)
print(C12List)

## 浮地比特与读取腔的耦合强度


In [ ]:
C12=47.82e-15   
C1=274.72e-15
C2=231.64e-15
R=10000/5.5
Cqr = 23.326e-15-0.219e-15
wr = 6.3e9


CJ=9.8e-15; C12 =C12+CJ*5.5
Capa = np.array([
    [C1,C12],
    [C12,C2],
])
elementParameter = [Capa,R,0]
FT = SingleFloatingTransmon(elementParameter)
el = FT.energyLevel
wq = el[1]*1e9

Cr = np.pi/4/50/2/np.pi/wr
Cq = (C1*C2+C1*C12+C2*C12)/(C1+C2)
Cc = Cqr/2
Cq = Cq-Cc
CqAll = Cq+Cc

Cqq = (Cq*Cr+Cq*Cc+Cr*Cc)/(Cr+Cc)
Crr = (Cq*Cr+Cq*Cc+Cr*Cc)/(Cq+Cc)
Ccc = (Cq*Cr+Cq*Cc+Cr*Cc)/(Cc)

g = np.sqrt(wq*wr)*np.sqrt(Cqq*Crr)/2/Ccc

print(wq/1e9)
print(g/1e6)

In [ ]:
from QubitEnergyFineSimulation import TwoFloatingTransmonWithGroundedCoupler
CJ=10e-15;     SRatio=5.3; 
C12=3e-15+CJ
C23=20.4e-15
C24=1.8e-15
C34=20.4e-15
C45=3e-15+CJ
C1=136e-15   
C2=136e-15   
C3=66e-15+CJ*SRatio
C4=C1
C5=C1
R=8500

# CJ=8e-15;     SRatio=5; 
# C12Raw=2.1e-15;
# C12 =C12Raw+CJ
# C23=20.6e-15
# C24=4.05e-15
# C34=20.6e-15
# C45=2.0e-15+CJ

# C1=110e-15
# C2=110e-15
# C3=60e-15+CJ*SRatio
# C4=C2
# C5=C1
# R=8500

Capa = np.array([
    [C1,C12,0,0,0],
    [C12,C2,C23,C24,0],
    [0,C23,C3,C34,0],
    [0,C24,C34,C4,C45],
    [0,0,0,C45,C5],
])
Rlist = [R,R/SRatio,R]
para = [Capa,Rlist]
FTWC = TwoFloatingTransmonWithGroundedCoupler(para)
el = FTWC.energyLevel
print(FTWC.Ec)

RlistWithoutC = [R,100,R]
paraWithoutC = [Capa,RlistWithoutC]
FTWithoutC = TwoFloatingTransmonWithGroundedCoupler(paraWithoutC)
elWithoutC = FTWithoutC.energyLevel


# print(el[0:15])
# print(elWithoutC[0:15])
couplerFrequency = el[FTWC.stateIndexList[3]]
qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
coulingXYQCQ = abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
couplingXYQQDirect = abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2
anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
deltaFreqency = qubitFreuqency-couplerFrequency
couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
print('Qubit Frequency:'+str(qubitFreuqency))
print('Coupler Frequency:'+str(couplerFrequency))
print('Anharmonicity:'+str(anharmonicity))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQ))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQ))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirect))
print('Qubit-Coupler couling:'+str(couplingQC))


## 浮地qubit-接地coupler-浮地qubit 1V4的系统能级

In [127]:
from QubitEnergyFineSimulation import FloatingTransmonWithGroundedCoupler1V4
# CJ=10e-15;     SRatio=6.5; 
# C12=5.728e-15+CJ
# C23=20.655e-15
# C24=1.568e-15
# C34=20.655e-15
# C45=5.728e-15+CJ
# C1=106.99e-15
# C2=C1  
# C3=63.36e-15+CJ*SRatio
# C4=C1
# C5=C1
# R=8200

CJ=9.8e-15;     SRatio=6.0; 
C12Raw=4.076e-15
C12 = C12Raw+CJ
C23=20.632e-15
C24=1.414e-15
C34=C23
C45=C12Raw+CJ

C1=157.21e-15-C12Raw-2*C24-C23-C34
C2=158.87e-15-C12Raw-2*C24-C23-C34
C3=138.67e-15+CJ-C23-C34
C4=C2
C5=C1
R=7800

# CJ=9.8e-15;     SRatio=6.5; 
# C12Raw=5.728e-15
# C12 = C12Raw+CJ
# C23=20.655e-15
# C24=1.568e-15
# C34=C23
# C45=C12

# C1=157.32e-15-C12Raw-2*C24-2*C23
# C2=157.32e-15-C12Raw-2*C24-2*C23
# C3=104.260e-15+CJ*SRatio-C23-C23
# C4=C2
# C5=C1
# R=8500

Capa = np.array([
    [C1,    C12,    0,      0,      0,      C23,    C23,    0,      0,      0,      0,      C24,    C24,    0,      0,      0,      0],
    [C12,   C2,     C23,    C24,    0,      0,      0,      C23,    0,      0,      0,      0,      0,      C24,    0,      0,      0],
    [0,     C23,    C3,     C23,    0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
    [0,     C24,    C23,    C4,     C45,    0,      0,      0,      C23,    0,      0,      0,      0,      0,      C24,    0,      0],
    [0,     0,      0,      C45,    C5,     0,      0,      0,      0,      C23,    C23,    0,      0,      0,      0,      C24,    C24],
    [C23,   0,      0,      0,      0,      C3,     0,      0,      0,      0,      0,      C23,    0,      0,      0,      0,      0],
    [C23,   0,      0,      0,      0,      0,      C3,     0,      0,      0,      0,      0,      C23,    0,      0,      0,      0],
    [0,     C23,    0,      0,      0,      0,      0,      C3,     0,      0,      0,      0,      0,      C23,    0,      0,      0],
    [0,     0,      0,      C23,    0,      0,      0,      0,      C3,     0,      0,      0,      0,      0,      C23,    0,      0],
    [0,     0,      0,      0,      C23,    0,      0,      0,      0,      C3,     0,      0,      0,      0,      0,      C23,    0],
    [0,     0,      0,      0,      C23,    0,      0,      0,      0,      0,      C3,     0,      0,      0,      0,      0,      C23],
    [0,     0,      0,      0,      0,      C23,    0,      0,      0,      0,      0,      C2,     0,      0,      0,      0,      0],
    [0,     0,      0,      0,      0,      0,      C23,    0,      0,      0,      0,      0,      C2,     0,      0,      0,     0],
    [0,     0,      0,      0,      0,      0,      0,      C23,    0,      0,      0,      0,      0,      C1,     0,      0,     0],
    [0,     0,      0,      0,      0,      0,      0,      0,      C23,    0,      0,      0,      0,      0,      C5,     0,      0],
    [0,     0,      0,      0,      0,      0,      0,      0,      0,      C23,    0,      0,      0,      0,      0,      C4,     0],
    [0,     0,      0,      0,      0,      0,      0,      0,      0,      0,      C23,    0,      0,      0,      0,      0,     C4],
    
])


Rlist = [R,R/SRatio,R,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R,R,R,R,R,R]
para = [Capa,Rlist]
FTWC = FloatingTransmonWithGroundedCoupler1V4(para)
el = FTWC.energyLevel

RlistWithoutC = [R,50,R,50,50,50,50,50,50,R,R,R,R,R,R]
paraWithoutC = [Capa,RlistWithoutC]
FTWithoutC = FloatingTransmonWithGroundedCoupler1V4(paraWithoutC)
elWithoutC = FTWithoutC.energyLevel


couplerFrequency = el[FTWC.stateIndexList[3]]
qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
couplingXYQQDirect = -abs(elWithoutC[2]-elWithoutC[1])/2 if FTWithoutC.couplingMinus else abs(elWithoutC[2]-elWithoutC[1])/2
anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
deltaFreqency = (qubitFreuqency-couplerFrequency)
couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
CouplerLeakage = FTWC.couplerLeakage

print(el[1:8])
print(elWithoutC[1:8])
print('Qubit Frequency:'+str(qubitFreuqency))
print('Coupler Frequency:'+str(couplerFrequency))
print('Anharmonicity:'+str(anharmonicity))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQ))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQ))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirect))
print('Qubit-Coupler couling:'+str(couplingQC))
print('CouplerLeakage:'+str(CouplerLeakage))

[ 5.34478875  5.3504534  10.45608391 10.45625904 10.65257942 10.69541275
 15.31792291]
[ 5.34516251  5.36543729 10.46773796 10.4693592  10.71221942 15.33386829
 15.33392635]
Qubit Frequency:5.347621071100727
Coupler Frequency:10.652579424981052
Anharmonicity:-0.23907066501396557
Qubit-Coupler-Qubit XY couling:0.002832323962647987
Qubit-Coupler-Qubit ZZ couling:0.00017060976980332043
Qubit-Qubit direct couling:0.010137390372720656
Qubit-Coupler couling:0.19685800232087483
CouplerLeakage:0.008483121934186957


### 系统能级随coupler频率变化关系

In [93]:
%%writefile FQGC.py
def func(flux,Capa,Rlist,couplingXYQQDirect):
    import numpy as np
    from QubitEnergyFineSimulation import FloatingTransmonWithGroundedCoupler1V4 
    RlistNew = Rlist.copy()
    RCoupler = Rlist[1]/np.cos(np.pi*flux)
    RlistNew[1] = RCoupler
    para = [Capa,RlistNew]
    FTWC = FloatingTransmonWithGroundedCoupler1V4(para)
    el = FTWC.energyLevel
    couplerFrequency = el[FTWC.stateIndexList[3]]
    qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
    qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
    coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
    anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
    coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
    deltaFreqency = (qubitFreuqency-couplerFrequency)
    couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
    result = {
        'couplerFrequency':couplerFrequency,
        'qubitFreuqency':qubitFreuqency,
        'el':el[0:8],
        'coulingXYQCQ':coulingXYQCQ,
        'coulingZZQCQ':coulingZZQCQ,
        'anharmonicity':anharmonicity,
        'couplingQC':couplingQC,
        'couplerLeakage':FTWC.couplerLeakage,
    }
    return(result)

Overwriting FQGC.py


In [94]:
import matplotlib.pyplot as plt
import FQGC
%matplotlib
fluxList = np.linspace(0.10,0.40,201)
freqList = []
couplingXYList = []
couplingZZList = []
anharmonicityList = []
CouplerLeakageList = []
   

numCores = int(mp.cpu_count())
pool = mp.Pool(numCores)
resultList = [pool.apply_async(FQGC.func, args=(flux,Capa,Rlist,couplingXYQQDirect)) for flux in fluxList]
pool.close()
pool.join()
resultList = [p.get() for p in resultList]

qubitFreqList = [result['qubitFreuqency'] for result in resultList]
couplerFreqList = [result['couplerFrequency'] for result in resultList]
freqList = [result['el'] for result in resultList]
couplingXYList = [result['coulingXYQCQ'] for result in resultList]
couplingZZList = [result['coulingZZQCQ'] for result in resultList]
anharmonicityList = [result['anharmonicity'] for result in resultList]
couplingQCList = [result['couplingQC'] for result in resultList]
CouplerLeakageList = [result['couplerLeakage'] for result in resultList]
  
freqList = np.transpose(freqList)

Using matplotlib backend: QtAgg


In [71]:
%matplotlib
plt.figure(1,dpi = 200)

for ii in range(len(freqList)):  
    plt.plot(fluxList,freqList[ii],label = str(ii))
plt.xlabel('flux');plt.ylabel('Frequency (GHz)');plt.legend()

plt.figure(2,dpi = 200)
plt.plot(fluxList,couplingXYList,label = 'XY')
plt.plot(fluxList,couplingZZList,label = 'ZZ')
plt.xlabel('flux');plt.ylabel('Coupling Strength (GHz)')
plt.legend()
 
plt.figure(3,dpi = 200)
plt.plot(fluxList,anharmonicityList)
plt.xlabel('flux');plt.ylabel('Anharmonicity (GHz)')
plt.tight_layout()
plt.show()

plt.figure(4,dpi = 200)
plt.plot(fluxList,CouplerLeakageList)
plt.xlabel('flux');plt.ylabel('CouplerLeakageList')
plt.tight_layout()
plt.show()

plt.figure(5,dpi = 200)
plt.plot(fluxList,couplingQCList)
plt.xlabel('flux');plt.ylabel('QC Coupling')
plt.tight_layout()
plt.show()

Using matplotlib backend: QtAgg


### 关断点和10M耦合情况


In [95]:
fluxRange = np.argmin(couplingXYList)
## coupling off
offIndex = np.argmin(np.abs(couplingXYList[0:fluxRange]))
offFlux = fluxList[offIndex]
offFreq = couplerFreqList[offIndex]
print([offFlux,offFreq,CouplerLeakageList[offIndex]*100])

## 耦合g

CZg = 0.016
coupling10Index = np.argmin(np.abs(np.array(couplingXYList[0:fluxRange])+CZg))
coupling10Flux = fluxList[coupling10Index]
coupling10Freq = couplerFreqList[coupling10Index]
print([CZg,coupling10Flux,coupling10Freq,CouplerLeakageList[coupling10Index]*100])


[0.20800000000000002, 9.082797774015996, 1.2327815876932444]
[0.016, 0.34750000000000003, 6.931500217259459, 4.412865399401576]


In [96]:
phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (freqList[1][phiIndex-1]+freqList[2][phiIndex-1]-freqList[1][phiIndex+1]-freqList[2][phiIndex+1])/2/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = freqList[1][0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000) 

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (freqList[1][phiIndex-1]+freqList[2][phiIndex-1]-freqList[1][phiIndex+1]-freqList[2][phiIndex+1])/2/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = freqList[1][0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000)

0.02724503100726139
2.2070044848288006


### Coupler T2导致的比特T2


#### 关断点

In [97]:
### 定义噪声强度和Coupler磁通偏置点
phi0 = 2.067833e-15;h = 6.62607015e-34;hbar = h/2/np.pi
AphiCouplerWhite = (1.05e-8*phi0)**2
AphiCouplerF = (46e-6*phi0)


phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
freqCoupler = couplerFreqList[phiIndex]

delta = qubitFreqList[phiIndex]-couplerFreqList[phiIndex]
gCoupling = couplingQCList[phiIndex]

### 计算coupler Tphi1和 Tphi2
slope = (couplerFreqList[phiIndex-1]-couplerFreqList[phiIndex+1])*2*np.pi*1e9/(fluxList[phiIndex-1]-fluxList[phiIndex+1])/phi0
gammaPhi1Coupler = AphiCouplerWhite*abs(slope)**2
gammaPhi2Coupler = AphiCouplerF*abs(slope)

TPhi1Coupler = 4/gammaPhi1Coupler*1e6
TPhi2Coupler = 1/gammaPhi2Coupler*1e6
print('Coupler在关断点，coupler磁通为{phi}Phi0的coupler Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Coupler,4),data2 = round(TPhi2Coupler,4)))
### 对比特T2影响
leakage = CouplerLeakageList[phiIndex]
TPhi1Qubit = TPhi1Coupler/leakage
TPhi2Qubit = TPhi2Coupler/leakage
print('Coupler在关断点，coupler磁通为{phi}Phi0的qubit Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Qubit,4),data2 = round(TPhi2Qubit,4)))

Coupler在关断点，coupler磁通为0.208Phi0的coupler Tphi1为7.6495 us, Tphi2为0.3157 us:
Coupler在关断点，coupler磁通为0.208Phi0的qubit Tphi1为620.5034 us, Tphi2为25.6053 us:


#### 做门点（10M）

In [98]:
### 定义噪声强度和Coupler磁通偏置点
phi0 = 2.067833e-15;h = 6.62607015e-34;hbar = h/2/np.pi
AphiCouplerWhite = (1.05e-8*phi0)**2
AphiCouplerF = (46e-6*phi0)

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
freqCoupler = couplerFreqList[phiIndex]


delta = qubitFreqList[phiIndex]-couplerFreqList[phiIndex]
gCoupling = couplingQCList[phiIndex]
### 计算coupler T2
slope = (couplerFreqList[phiIndex-1]-couplerFreqList[phiIndex+1])*2*np.pi*1e9/(fluxList[phiIndex-1]-fluxList[phiIndex+1])/phi0

gammaPhi1Coupler = AphiCouplerWhite*abs(slope)**2
gammaPhi2Coupler = AphiCouplerF*abs(slope)

TPhi1Coupler = 4/gammaPhi1Coupler*1e6
TPhi2Coupler = 1/gammaPhi2Coupler*1e6
print('Coupler在CZ做门点，coupler磁通为{phi}Phi0的coupler Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Coupler,4),data2 = round(TPhi2Coupler,4)))
### 对比特T2影响
leakage = CouplerLeakageList[phiIndex]
TPhi1Qubit = TPhi1Coupler/leakage
TPhi2Qubit = TPhi2Coupler/leakage
print('Coupler在CZ做门点，coupler磁通为{phi}Phi0的qubit Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Qubit,4),data2 = round(TPhi2Qubit,4)))

### 做门错误
tgate = 1/2/np.sqrt(2)/0.8/CZg
errorPhi1 = 2*1/2*(tgate/(TPhi1Qubit*1000))
errorPhi2 = 2*1/2*(tgate/(TPhi2Qubit*1000))**2

errorCZ = errorPhi1+errorPhi2
print('Coupler在CZ做门点，做门时间为{time}ns，coupler引入的错误为：{data}%,其中白噪声引入的错误为：{error1}%，1/f噪声引入的错误为：{error2}%:'.format(time = round(tgate,1),data = round(errorCZ*100,4),error1 = round(errorPhi1*100,4),error2 = round(errorPhi2*100,4)))

Coupler在CZ做门点，coupler磁通为0.348Phi0的coupler Tphi1为2.1976 us, Tphi2为0.1692 us:
Coupler在CZ做门点，coupler磁通为0.348Phi0的qubit Tphi1为49.7999 us, Tphi2为3.834 us:
Coupler在CZ做门点，做门时间为27.6ns，coupler引入的错误为：0.0607%,其中白噪声引入的错误为：0.0555%，1/f噪声引入的错误为：0.0052%:


### Coupler热激发导致的dephasing


In [100]:
def DephasingCouplerThermal(phiSet):
    T1Ccoupler = 5e-6
    T_est = 60e-3
    phiIndex = np.argmin(abs(fluxList-phiSet))

    wc = freqList[3][phiIndex]*2*np.pi*1e9
    wq = (freqList[1][phiIndex]+freqList[2][phiIndex])/2*2*np.pi*1e9
    hbar=1.054560652926899e-34;     
    h = hbar*2*np.pi;     
    e = 1.60217662e-19;     
    phi0 = h/2/e
    kb = 1.380649e-23
    R = 50
    
    
    nc = 1/(np.exp(hbar*wc/kb/T_est)-1)
    chi = -(couplingQCList[phiIndex]*1e9*2*np.pi)**2/(wq-wc)*1/(1+(wq-wc)/(anharmonicityList[phiIndex]*1e9*2*np.pi))
    kb = 1.380649e-23;h = 6.62607015e-34;hbar = h/2/np.pi
    kappa = 1/T1Ccoupler

    eta = kappa**2/(kappa**2+4*chi**2)
    gammaPhi = eta*4*chi**2/kappa*nc
    Tphi = 1/gammaPhi
    errorPhi1 = 2*1/2*(tgate/1e9/(Tphi))
    print(nc,tgate,Tphi/1e-6,errorPhi1*100)




phiSet = offFlux
DephasingCouplerThermal(phiSet)
phiSet = coupling10Flux
DephasingCouplerThermal(phiSet)







0.0007000802172806323 27.62135864009951 7147.97841347872 0.0003864219649574595
0.003925154499857881 27.62135864009951 1273.874439195791 0.0021682952251979508


## 浮地qubit-浮地coupler-浮地qubit 1V4的系统能级

In [13]:
from QubitEnergyFineSimulation import FloatingTransmonWithFloatingCoupler1V4


CJ=9.8e-15;     SRatio=2; 
C12Raw=1.59e-15; C12 = C12Raw+CJ
C23=22.045e-15
C24=0.0e-15
C34Raw=57.940e-15; C34 = C34Raw+CJ*SRatio
C35=C24
C45=C23
C56=C12

C1=150.309e-15-C12Raw-2*C23-2*C24
C2=154.912e-15-C12Raw-2*C23-2*C24
C3=200.727e-15-C23-C34Raw-C35
C4=200.371e-15-C24-C34Raw-C45
C5=C2
C6=C1
R=9000



Capa = np.array([
    [C1,    C12,    0,      0,      0,      0,      C23,    C24,    C23,    C24,    0,     0,    0,      0,      0,      0,     0,      0],
    [C12,   C2,     C23,    C24,    0,      0,      0,      0,      0,      0,      C23,   C24,  0,      0,      0,      0,     0,      0],
    [0,     C23,    C3,     C34,    C35,    0,      0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [0,     C24,    C34,    C4,     C45,    0,      0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [0,     0,      C35,    C45,    C5,     C56,    0,      0,      0,      0,      0,     0,    C35,    C45,    0,      0,     0,      0],
    [0,     0,      0,      0,      C56,    C6,     0,      0,      0,      0,      0,     0,    0,      0,      C35,    C45,   C35,    C45,],
    [C23,   0,      0,      0,      0,      0,      C3,     C34,    0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [C24,   0,      0,      0,      0,      0,      C34,    C4,     0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [C23,   0,      0,      0,      0,      0,      0,      0,      C3,     C34,    0,     0,    0,      0,      0,      0,     0,      0],
    [C24,   0,      0,      0,      0,      0,      0,      0,      C34,    C4,     0,     0,    0,      0,      0,      0,     0,      0],
    [0,     C23,    0,      0,      0,      0,      0,      0,      0,      0,      C3,    C34,  0,      0,      0,      0,     0,      0],
    [0,     C24,    0,      0,      0,      0,      0,      0,      0,      0,      C34,   C4,   0,      0,      0,      0,     0,      0],
    [0,     0,      0,      0,      C35,    0,      0,      0,      0,      0,      0,     0,    C3,     C34,    0,      0,     0,      0],
    [0,     0,      0,      0,      C45,    0,      0,      0,      0,      0,      0,     0,    C34,    C4,     0,      0,     0,      0],
    [0,     0,      0,      0,      0,      C35,    0,      0,      0,      0,      0,     0,    0,      0,      C3,     C34,   0,      0],
    [0,     0,      0,      0,      0,      C45,    0,      0,      0,      0,      0,     0,    0,      0,      C34,    C4,    0,      0],
    [0,     0,      0,      0,      0,      C35,    0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     C3,     C34],
    [0,     0,      0,      0,      0,      C45,    0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     C34,    C4],
])
Rlist = [R,R/SRatio,R,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio]
para = [Capa,Rlist]
FTWC = FloatingTransmonWithFloatingCoupler1V4(para)
el = FTWC.energyLevel
RlistWithoutC = [R,50,R,50,50,50,50,50,50]
paraWithoutC = [Capa,RlistWithoutC]
FTWithoutC = FloatingTransmonWithFloatingCoupler1V4(paraWithoutC)
elWithoutC = FTWithoutC.energyLevel


couplerFrequency = el[FTWC.stateIndexList[3]]
qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
couplingXYQQDirect = -abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2 if FTWithoutC.couplingMinus else abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2
anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
deltaFreqency = (qubitFreuqency-couplerFrequency)
couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
CouplerLeakage = FTWC.couplerLeakage

print(el[1:8])
print(elWithoutC[1:8])
print('Qubit Frequency:'+str(qubitFreuqency))
print('Coupler Frequency:'+str(couplerFrequency))
print('Anharmonicity:'+str(anharmonicity))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQ))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQ))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirect))
print('Qubit-Coupler couling:'+str(couplingQC))
print('CouplerLeakage:'+str(CouplerLeakage))

[ 5.01894884  5.09435872  5.62685744  9.86128282  9.87706189 10.12721975
 10.62965843]
[ 5.06975757  5.09452034  9.90630014  9.90856557 10.16653424 14.47098204
 14.47123346]
Qubit Frequency:5.05665377940821
Coupler Frequency:5.626857440692862
Anharmonicity:-0.24413520419586732
Qubit-Coupler-Qubit XY couling:0.03770494395102908
Qubit-Coupler-Qubit ZZ couling:0.013912193493398561
Qubit-Qubit direct couling:0.0
Qubit-Coupler couling:nan
CouplerLeakage:0.09778725028595481


C:\Users\Chen\AppData\Local\Temp\ipykernel_11320\803279894.py:60: RuntimeWarning: invalid value encountered in sqrt
  couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)


### 系统能级随coupler频率变化关系


In [6]:
%%writefile FQFC.py
def func(flux,Capa,Rlist):
    import numpy as np
    from QubitEnergyFineSimulation import FloatingTransmonWithFloatingCoupler1V4 
    RlistNew = Rlist.copy()
    RCoupler = Rlist[1]/np.cos(np.pi*flux)
    RlistNew[1] = RCoupler
    para = [Capa,RlistNew]
    FTWC = FloatingTransmonWithFloatingCoupler1V4(para)
    el = FTWC.energyLevel
    couplerFrequency = el[FTWC.stateIndexList[3]]
    qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
    qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
    coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
    anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
    coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
    deltaFreqency = (qubitFreuqency-couplerFrequency)
    couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
    result = {
        'couplerFrequency':couplerFrequency,
        'qubitFreuqency':qubitFreuqency,
        'el':el[0:8],
        'coulingXYQCQ':coulingXYQCQ,
        'coulingZZQCQ':coulingZZQCQ,
        'anharmonicity':anharmonicity,
        'couplingQC':couplingQC,
        'couplerLeakage':FTWC.couplerLeakage,
    }
    return(result)

Overwriting FQFC.py


In [9]:
import matplotlib.pyplot as plt
import FQFC
%matplotlib
fluxList = np.linspace(0.00,0.40,101)
qubitFreqList = []
couplerFreqList = []
freqList = []
couplingXYList = []
couplingZZList = []
anharmonicityList = []
CouplerLeakageList = []


numCores = int(mp.cpu_count())
pool = mp.Pool(numCores)
resultList = [pool.apply_async(FQFC.func, args=(flux,Capa,Rlist,)) for flux in fluxList]
pool.close()
pool.join()
resultList = [p.get() for p in resultList]

qubitFreqList = [result['qubitFreuqency'] for result in resultList]
couplerFreqList = [result['couplerFrequency'] for result in resultList]
freqList = [result['el'] for result in resultList]
couplingXYList = [result['coulingXYQCQ'] for result in resultList]
couplingZZList = [result['coulingZZQCQ'] for result in resultList]
anharmonicityList = [result['anharmonicity'] for result in resultList]
CouplerLeakageList = [result['couplerLeakage'] for result in resultList]
  
freqList = np.transpose(freqList)

Using matplotlib backend: QtAgg


In [14]:
%matplotlib
plt.figure(1,dpi = 200)

for ii in range(len(freqList)):  
    plt.plot(fluxList,freqList[ii],label = str(ii))
plt.xlabel('flux');plt.ylabel('Frequency (GHz)');plt.legend()

plt.figure(2,dpi = 200)
plt.plot(fluxList,couplingXYList,label = 'XY')
plt.plot(fluxList,couplingZZList,label = 'ZZ')
plt.xlabel('flux');plt.ylabel('Coupling Strength (GHz)')
plt.legend()
 
plt.figure(3,dpi = 200)
plt.plot(fluxList,anharmonicityList)
plt.xlabel('flux');plt.ylabel('Anharmonicity (GHz)')
plt.tight_layout()
plt.show()

plt.figure(4,dpi = 200)
plt.plot(fluxList,CouplerLeakageList)
plt.xlabel('flux');plt.ylabel('CouplerLeakageList')
plt.tight_layout()
plt.show()

plt.figure(5,dpi = 200)
plt.plot(fluxList,couplingQCList)
plt.xlabel('flux');plt.ylabel('QC Coupling')
plt.tight_layout()
plt.show()

### 关断时比特和coupler频率

In [15]:
fluxRange = np.argmin(couplingXYList)
## coupling off
offIndex = np.argmin(np.abs(couplingXYList[0:fluxRange]))
offFlux = fluxList[offIndex]
offFreq = couplerFreqList[offIndex]
print([offFlux,offFreq,CouplerLeakageList[offIndex]*100])

## 耦合g

CZg = 0.016
coupling10Index = np.argmin(np.abs(np.array(couplingXYList[0:fluxRange])+CZg))
coupling10Flux = fluxList[coupling10Index]
coupling10Freq = couplerFreqList[coupling10Index]
print([coupling10Flux,coupling10Freq,CouplerLeakageList[coupling10Index]*100])

[0.17200000000000001, 5.288950714359081, 40.44130010577802]
[0.17200000000000001, 5.288950714359081, 40.44130010577802]


In [18]:
offIndex

43

In [ ]:
phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (freqList[1][phiIndex-1]+freqList[2][phiIndex-1]-freqList[1][phiIndex+1]-freqList[2][phiIndex+1])/2/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = freqList[1][0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000) 

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (freqList[1][phiIndex-1]+freqList[2][phiIndex-1]-freqList[1][phiIndex+1]-freqList[2][phiIndex+1])/2/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = freqList[1][0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000)

### Coupler T2导致的比特T2


#### 关断点

In [ ]:
### 定义噪声强度和Coupler磁通偏置点
phi0 = 2.067833e-15;h = 6.62607015e-34;hbar = h/2/np.pi
AphiCouplerWhite = (1.05e-8*phi0)**2
AphiCouplerF = (46e-6*phi0)


phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
freqCoupler = freqList[3][phiIndex]

delta = (freqList[1][phiIndex]+freqList[2][phiIndex])/2-freqList[3][phiIndex]
gCoupling = couplingQC

### 计算coupler Tphi1和 Tphi2
slope = (freqList[3][phiIndex-1]-freqList[3][phiIndex+1])*2*np.pi*1e9/(fluxList[phiIndex-1]-fluxList[phiIndex+1])/phi0
gammaPhi1Coupler = AphiCouplerWhite*abs(slope)**2
gammaPhi2Coupler = AphiCouplerF*abs(slope)

TPhi1Coupler = 4/gammaPhi1Coupler*1e6
TPhi2Coupler = 1/gammaPhi2Coupler*1e6
print('Coupler在关断点，coupler磁通为{phi}Phi0的coupler Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Coupler,4),data2 = round(TPhi2Coupler,4)))
### 对比特T2影响
leakage = CouplerLeakageList[phiIndex]
TPhi1Qubit = TPhi1Coupler/leakage
TPhi2Qubit = TPhi2Coupler/leakage
print('Coupler在关断点，coupler磁通为{phi}Phi0的qubit Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Qubit,4),data2 = round(TPhi2Qubit,4)))

Coupler在关断点，coupler磁通为0.1Phi0的coupler Tphi1为6.8755 us, Tphi2为0.2993 us:
Coupler在关断点，coupler磁通为0.1Phi0的qubit Tphi1为509.0349 us, Tphi2为22.1563 us:


#### 做门点（10M）

In [ ]:
### 定义噪声强度和Coupler磁通偏置点
phi0 = 2.067833e-15;h = 6.62607015e-34;hbar = h/2/np.pi
AphiCouplerWhite = (1.05e-8*phi0)**2
AphiCouplerF = (46e-6*phi0)

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
freqCoupler = freqList[3][phiIndex]


delta = (freqList[1][phiIndex]+freqList[2][phiIndex])/2-freqList[3][phiIndex]
gCoupling = couplingQC
### 计算coupler T2
slope = (freqList[3][phiIndex-1]-freqList[3][phiIndex+1])*2*np.pi*1e9/(fluxList[phiIndex-1]-fluxList[phiIndex+1])/phi0

gammaPhi1Coupler = AphiCouplerWhite*abs(slope)**2
gammaPhi2Coupler = AphiCouplerF*abs(slope)

TPhi1Coupler = 4/gammaPhi1Coupler*1e6
TPhi2Coupler = 1/gammaPhi2Coupler*1e6
print('Coupler在CZ做门点，coupler磁通为{phi}Phi0的coupler Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Coupler,4),data2 = round(TPhi2Coupler,4)))
### 对比特T2影响
leakage = CouplerLeakageList[phiIndex]
TPhi1Qubit = TPhi1Coupler/leakage
TPhi2Qubit = TPhi2Coupler/leakage
print('Coupler在CZ做门点，coupler磁通为{phi}Phi0的qubit Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Qubit,4),data2 = round(TPhi2Qubit,4)))

### 做门错误
tgate = 1/2/np.sqrt(2)/0.8/CZg
errorPhi1 = 2*1/2*(tgate/(TPhi1Qubit*1000))
errorPhi2 = 2*1/2*(tgate/(TPhi2Qubit*1000))**2

errorCZ = errorPhi1+errorPhi2
print('Coupler在CZ做门点，做门时间为{time}ns，coupler引入的错误为：{data}%,其中白噪声引入的错误为：{error1}%，1/f噪声引入的错误为：{error2}%:'.format(time = round(tgate,1),data = round(errorCZ*100,4),error1 = round(errorPhi1*100,4),error2 = round(errorPhi2*100,4)))

Coupler在CZ做门点，coupler磁通为0.304Phi0的coupler Tphi1为4.1034 us, Tphi2为0.2312 us:
Coupler在CZ做门点，coupler磁通为0.304Phi0的qubit Tphi1为91.4711 us, Tphi2为5.1537 us:
Coupler在CZ做门点，做门时间为27.6ns，coupler引入的错误为：0.0331%,其中白噪声引入的错误为：0.0302%，1/f噪声引入的错误为：0.0029%:


## 浮地qubit-浮地coupler-浮地qubit 1V4的系统能级(Coupler在下)

In [149]:
from QubitEnergyFineSimulation import FloatingTransmonWithFloatingCoupler1V4


CJ=9.8e-15;     SRatio=1.4; 
C12Raw=1.59e-15; C12 = C12Raw+CJ
C23=19.045e-15
C24=0.0e-15
C34Raw=40.940e-15; C34 = C34Raw+CJ*SRatio
C35=C24
C45=C23
C56=C12

C1=147.309e-15-C12Raw-2*C23-2*C24
C2=147.912e-15-C12Raw-2*C23-2*C24
C3=160.727e-15-C23-C34Raw-C35
C4=160.371e-15-C24-C34Raw-C45
C5=C2
C6=C1
R=9000


# CJ=9.8e-15;     SRatio=1.0; 
# C12 = 46e-15
# C23=19.5e-15
# C24=2e-15
# C34 = 61e-15
# C35=C24
# C45=C23
# C56=C12

# C1=110e-15
# C2=110e-15
# C3=80e-15
# C4=80e-15
# C5=C2
# C6=C1
# R=9500



Capa = np.array([
    [C1,    C12,    0,      0,      0,      0,      C23,    C24,    C23,    C24,    0,     0,    0,      0,      0,      0,     0,      0],
    [C12,   C2,     C23,    C24,    0,      0,      0,      0,      0,      0,      C23,   C24,  0,      0,      0,      0,     0,      0],
    [0,     C23,    C3,     C34,    C35,    0,      0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [0,     C24,    C34,    C4,     C45,    0,      0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [0,     0,      C35,    C45,    C5,     C56,    0,      0,      0,      0,      0,     0,    C35,    C45,    0,      0,     0,      0],
    [0,     0,      0,      0,      C56,    C6,     0,      0,      0,      0,      0,     0,    0,      0,      C35,    C45,   C35,    C45,],
    [C23,   0,      0,      0,      0,      0,      C3,     C34,    0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [C24,   0,      0,      0,      0,      0,      C34,    C4,     0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [C23,   0,      0,      0,      0,      0,      0,      0,      C3,     C34,    0,     0,    0,      0,      0,      0,     0,      0],
    [C24,   0,      0,      0,      0,      0,      0,      0,      C34,    C4,     0,     0,    0,      0,      0,      0,     0,      0],
    [0,     C23,    0,      0,      0,      0,      0,      0,      0,      0,      C3,    C34,  0,      0,      0,      0,     0,      0],
    [0,     C24,    0,      0,      0,      0,      0,      0,      0,      0,      C34,   C4,   0,      0,      0,      0,     0,      0],
    [0,     0,      0,      0,      C35,    0,      0,      0,      0,      0,      0,     0,    C3,     C34,    0,      0,     0,      0],
    [0,     0,      0,      0,      C45,    0,      0,      0,      0,      0,      0,     0,    C34,    C4,     0,      0,     0,      0],
    [0,     0,      0,      0,      0,      C35,    0,      0,      0,      0,      0,     0,    0,      0,      C3,     C34,   0,      0],
    [0,     0,      0,      0,      0,      C45,    0,      0,      0,      0,      0,     0,    0,      0,      C34,    C4,    0,      0],
    [0,     0,      0,      0,      0,      C35,    0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     C3,     C34],
    [0,     0,      0,      0,      0,      C45,    0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     C34,    C4],
])
Rlist = [R,R/SRatio,R,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio]
para = [Capa,Rlist]
FTWC = FloatingTransmonWithFloatingCoupler1V4(para)
el = FTWC.energyLevel
RlistWithoutC = [R,50,R,50,50,50,50,50,50]
paraWithoutC = [Capa,RlistWithoutC]
FTWithoutC = FloatingTransmonWithFloatingCoupler1V4(paraWithoutC)
elWithoutC = FTWithoutC.energyLevel


couplerFrequency = el[FTWC.stateIndexList[3]]
qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
couplingXYQQDirect = -abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2 if FTWithoutC.couplingMinus else abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2
anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
deltaFreqency = (qubitFreuqency-couplerFrequency)
couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
CouplerLeakage = FTWC.couplerLeakage

print(el[1:8])
print(elWithoutC[1:8]) 
print('Qubit Frequency:'+str(qubitFreuqency)) 
print('Coupler Frequency:'+str(couplerFrequency))
print('Anharmonicity:'+str(anharmonicity))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQ))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQ))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirect))
print('Qubit-Coupler couling:'+str(couplingQC))
print('CouplerLeakage:'+str(CouplerLeakage))
print(FTWC.Ec)
print(FTWC.Ej)

[ 5.01708202  5.1596842   5.39527143  9.92895668  9.9740976  10.19333445
 10.39459784]
[ 5.13630253  5.15986529 10.03098111 10.03297944 10.29815365 14.64635473
 14.6466432 ]
Qubit Frequency:5.088383109207517
Coupler Frequency:5.395271427644681
Anharmonicity:-0.20266862096166172
Qubit-Coupler-Qubit XY couling:0.07130109003927476
Qubit-Coupler-Qubit ZZ couling:0.21783162133522005
Qubit-Qubit direct couling:-0.011781377108590174
Qubit-Coupler couling:nan
CouplerLeakage:0.3388737231527517
[[ 1.48298798  0.06116377  0.0032564 ]
 [ 0.06116377  1.07429848 -0.06135137]
 [ 0.0032564  -0.06135137  1.48307308]]
[[ 97.09014201   0.           0.        ]
 [  0.         135.92619881   0.        ]
 [  0.           0.          97.09014201]]


C:\Users\Chen\AppData\Local\Temp\ipykernel_44232\2480319454.py:78: RuntimeWarning: invalid value encountered in sqrt
  couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)


### 系统能级随coupler频率变化关系


In [150]:
%%writefile FQFC.py
def func(flux,Capa,Rlist,couplingXYQQDirect):
    import numpy as np
    from QubitEnergyFineSimulation import FloatingTransmonWithFloatingCoupler1V4 
    RlistNew = Rlist.copy()
    RCoupler = Rlist[1]/np.cos(np.pi*flux)
    RlistNew[1] = RCoupler
    para = [Capa,RlistNew]
    FTWC = FloatingTransmonWithFloatingCoupler1V4(para)
    el = FTWC.energyLevel
    couplerFrequency = el[FTWC.stateIndexList[3]]
    qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
    qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
    coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
    anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
    coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
    deltaFreqency = (qubitFreuqency-couplerFrequency)
    couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
    result = {
        'couplerFrequency':couplerFrequency,
        'qubitFreuqency':qubitFreuqency,
        'el':el[0:8],
        'coulingXYQCQ':coulingXYQCQ,
        'coulingZZQCQ':coulingZZQCQ,
        'anharmonicity':anharmonicity,
        'couplingQC':couplingQC,
        'couplerLeakage':FTWC.couplerLeakage,
    }
    return(result)

Overwriting FQFC.py


In [151]:
import matplotlib.pyplot as plt
import FQFC
fluxList = np.linspace(0.00,0.40,201)
qubitFreqList = []
couplerFreqList = []
freqList = []
couplingXYList = []
couplingZZList = []
anharmonicityList = []
CouplerLeakageList = []


numCores = int(mp.cpu_count())
pool = mp.Pool(numCores)
resultList = [pool.apply_async(FQFC.func, args=(flux,Capa,Rlist,couplingXYQQDirect)) for flux in fluxList]
pool.close()
pool.join()
resultList = [p.get() for p in resultList]

qubitFreqList = [result['qubitFreuqency'] for result in resultList]
couplerFreqList = [result['couplerFrequency'] for result in resultList]
freqList = [result['el'] for result in resultList]
couplingXYList = [result['coulingXYQCQ'] for result in resultList]
couplingZZList = [result['coulingZZQCQ'] for result in resultList]
anharmonicityList = [result['anharmonicity'] for result in resultList]
CouplerLeakageList = [result['couplerLeakage'] for result in resultList]
couplingQCList = [result['couplingQC'] for result in resultList]
   
freqList = np.transpose(freqList)

In [145]:
%matplotlib
plt.figure(1,dpi = 200)

for ii in range(len(freqList)):  
    plt.plot(fluxList,freqList[ii],label = str(ii))
plt.xlabel('flux');plt.ylabel('Frequency (GHz)');plt.legend()

plt.figure(2,dpi = 200)
plt.plot(fluxList,couplingXYList,label = 'XY')
plt.plot(fluxList,couplingZZList,label = 'ZZ')
plt.xlabel('flux');plt.ylabel('Coupling Strength (GHz)')
plt.legend()
 
plt.figure(3,dpi = 200)
plt.plot(fluxList,anharmonicityList)
plt.xlabel('flux');plt.ylabel('Anharmonicity (GHz)')
plt.tight_layout()
plt.show()

plt.figure(4,dpi = 200)
plt.plot(fluxList,CouplerLeakageList)
plt.xlabel('flux');plt.ylabel('CouplerLeakageList')
plt.tight_layout()
plt.show()

plt.figure(5,dpi = 200)
plt.plot(fluxList,couplingQCList)
plt.xlabel('flux');plt.ylabel('QC Coupling')
plt.tight_layout()
plt.show()

Using matplotlib backend: QtAgg


### 关断时比特和coupler频率

In [152]:
fluxRange = np.argmax(couplingXYList)
fluxListNew = fluxList[fluxRange:]
couplerFreqListNew = couplerFreqList[fluxRange:]
CouplerLeakageListNew = CouplerLeakageList[fluxRange:]
## coupling off
offIndex = np.argmin(np.abs(couplingXYList[fluxRange:]))
offFlux = fluxListNew[offIndex]
offFreq = couplerFreqListNew[offIndex]
print([offFlux,offFreq,CouplerLeakageListNew[offIndex]*100])

## 耦合g

CZg = 0.016
coupling10Index = np.argmin(np.abs(np.array(couplingXYList[fluxRange:])-CZg))
coupling10Flux = fluxListNew[coupling10Index]
coupling10Freq = couplerFreqListNew[coupling10Index]
print([coupling10Flux,coupling10Freq,CouplerLeakageListNew[coupling10Index]*100])

[0.342, 3.5687226133306877, 0.46465643978253096]
[0.222, 4.543787119630974, 7.131357035533083]


In [153]:
RlistOff = [R,R/SRatio/np.cos(np.pi*offFlux),R,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio]
paraOff = [Capa,RlistOff]
FTWCOff = FloatingTransmonWithFloatingCoupler1V4(paraOff)
elOff = FTWCOff.energyLevel



couplerFrequencyOff = elOff[FTWCOff.stateIndexList[3]]
qubitFreuqencyOff = (elOff[FTWCOff.stateIndexList[2]]+elOff[FTWCOff.stateIndexList[1]])/2
qubit02FreuqencyOff = (elOff[FTWCOff.stateIndexList[5]]+elOff[FTWCOff.stateIndexList[6]])/2
coulingXYQCQOff = -abs(elOff[FTWCOff.stateIndexList[2]]-elOff[FTWCOff.stateIndexList[1]])/2 if FTWCOff.couplingMinus else abs(elOff[FTWCOff.stateIndexList[2]]-elOff[FTWCOff.stateIndexList[1]])/2
couplingXYQQDirectOff = -abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2 if FTWithoutC.couplingMinus else abs(elWithoutC[FTWithoutC.stateIndexList[2]]-elWithoutC[FTWithoutC.stateIndexList[1]])/2
anharmonicityOff = (qubit02FreuqencyOff-2*qubitFreuqencyOff)
deltaFreqencyOff = (qubitFreuqencyOff-couplerFrequencyOff)
couplingQCOff = np.sqrt((coulingXYQCQOff-couplingXYQQDirectOff)*deltaFreqencyOff)
coulingZZQCQOff = abs(elOff[FTWCOff.stateIndexList[2]]+elOff[FTWCOff.stateIndexList[1]]-elOff[FTWCOff.stateIndexList[4]])
CouplerLeakageOff = FTWCOff.couplerLeakage

print(elOff[1:8])
print(elWithoutC[1:8])
print('Qubit Frequency:'+str(qubitFreuqencyOff))
print('Coupler Frequency:'+str(couplerFrequencyOff))
print('Anharmonicity:'+str(anharmonicityOff))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQOff))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQOff))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirectOff))
print('Qubit-Coupler couling:'+str(couplingQCOff))
print('CouplerLeakage:'+str(CouplerLeakageOff))

[ 3.56872261  5.15957638  5.15975135  7.14775877  8.72651187  8.72681634
 10.05725883]
[ 5.13630253  5.15986529 10.03098111 10.03297944 10.29815365 14.64635473
 14.6466432 ]
Qubit Frequency:5.159663866961061
Coupler Frequency:3.5687226133306877
Anharmonicity:-0.2618966762465824
Qubit-Coupler-Qubit XY couling:8.748514978629274e-05
Qubit-Coupler-Qubit ZZ couling:7.756269031489182e-07
Qubit-Qubit direct couling:-0.011781377108590174
Qubit-Coupler couling:0.1374142008691521
CouplerLeakage:0.00464656439782531


In [132]:
phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (qubitFreqList[phiIndex-1]-qubitFreqList[phiIndex+1])/2/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = qubitFreqList[0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000) 

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
slope = (qubitFreqList[phiIndex-1]-qubitFreqList[phiIndex+1])/2/(fluxList[phiIndex-1]-fluxList[phiIndex+1])
w0 = qubitFreqList[0]
fList = np.linspace(0,0.49,1001)
SlopeList = [-np.pi/2*w0*np.sin(np.pi*f)/np.sqrt(np.cos(np.pi*f)) for f in fList]
SlopeIndex = np.argmin(abs(slope-SlopeList))
deltaFreq = w0-w0*np.sqrt(np.cos(np.pi*fList[SlopeIndex]))
print(deltaFreq*1000)

0.04825461261237507
0.36492929159415155


### Coupler T2导致的比特T2


#### 关断点

In [113]:
### 定义噪声强度和Coupler磁通偏置点
phi0 = 2.067833e-15;h = 6.62607015e-34;hbar = h/2/np.pi
AphiCouplerWhite = (1.05e-8*phi0)**2
AphiCouplerF = (46e-6*phi0)


phiSet = offFlux
phiIndex = np.argmin(abs(fluxList-phiSet))
freqCoupler = couplerFreqList[phiIndex]

delta = qubitFreqList[phiIndex]-couplerFreqList[phiIndex]
gCoupling = couplingQCList[phiIndex]

### 计算coupler Tphi1和 Tphi2
slope = (couplerFreqList[phiIndex-1]-couplerFreqList[phiIndex+1])*2*np.pi*1e9/(fluxList[phiIndex-1]-fluxList[phiIndex+1])/phi0
gammaPhi1Coupler = AphiCouplerWhite*abs(slope)**2
gammaPhi2Coupler = AphiCouplerF*abs(slope)

TPhi1Coupler = 4/gammaPhi1Coupler*1e6
TPhi2Coupler = 1/gammaPhi2Coupler*1e6
print('Coupler在关断点，coupler磁通为{phi}Phi0的coupler Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Coupler,4),data2 = round(TPhi2Coupler,4)))
### 对比特T2影响
leakage = CouplerLeakageList[phiIndex]
TPhi1Qubit = TPhi1Coupler/leakage
TPhi2Qubit = TPhi2Coupler/leakage
print('Coupler在关断点，coupler磁通为{phi}Phi0的qubit Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Qubit,4),data2 = round(TPhi2Qubit,4)))

Coupler在关断点，coupler磁通为0.292Phi0的coupler Tphi1为13.0081 us, Tphi2为0.4116 us:
Coupler在关断点，coupler磁通为0.292Phi0的qubit Tphi1为1155.1858 us, Tphi2为36.555 us:


#### 做门点（10M）

In [88]:
### 定义噪声强度和Coupler磁通偏置点
phi0 = 2.067833e-15;h = 6.62607015e-34;hbar = h/2/np.pi
AphiCouplerWhite = (1.05e-8*phi0)**2
AphiCouplerF = (46e-6*phi0)

phiSet = coupling10Flux
phiIndex = np.argmin(abs(fluxList-phiSet))
freqCoupler = couplerFreqList[phiIndex]


delta = qubitFreqList[phiIndex]-couplerFreqList[phiIndex]
gCoupling = couplingQCList[phiIndex]
### 计算coupler T2
slope = (couplerFreqList[phiIndex-1]-couplerFreqList[phiIndex+1])*2*np.pi*1e9/(fluxList[phiIndex-1]-fluxList[phiIndex+1])/phi0

gammaPhi1Coupler = AphiCouplerWhite*abs(slope)**2
gammaPhi2Coupler = AphiCouplerF*abs(slope)

TPhi1Coupler = 4/gammaPhi1Coupler*1e6
TPhi2Coupler = 1/gammaPhi2Coupler*1e6
print('Coupler在CZ做门点，coupler磁通为{phi}Phi0的coupler Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Coupler,4),data2 = round(TPhi2Coupler,4)))
### 对比特T2影响
leakage = CouplerLeakageList[phiIndex]
TPhi1Qubit = TPhi1Coupler/leakage
TPhi2Qubit = TPhi2Coupler/leakage
print('Coupler在CZ做门点，coupler磁通为{phi}Phi0的qubit Tphi1为{data1} us, Tphi2为{data2} us:'.format(phi = round(phiSet,3),data1 = round(TPhi1Qubit,4),data2 = round(TPhi2Qubit,4)))

### 做门错误
tgate = 1/2/np.sqrt(2)/0.8/CZg
errorPhi1 = 2*1/2*(tgate/(TPhi1Qubit*1000))
errorPhi2 = 2*1/2*(tgate/(TPhi2Qubit*1000))**2

errorCZ = errorPhi1+errorPhi2
print('Coupler在CZ做门点，做门时间为{time}ns，coupler引入的错误为：{data}%,其中白噪声引入的错误为：{error1}%，1/f噪声引入的错误为：{error2}%:'.format(time = round(tgate,1),data = round(errorCZ*100,4),error1 = round(errorPhi1*100,4),error2 = round(errorPhi2*100,4)))

Coupler在CZ做门点，coupler磁通为0.196Phi0的coupler Tphi1为39.3081 us, Tphi2为0.7156 us:
Coupler在CZ做门点，coupler磁通为0.196Phi0的qubit Tphi1为420.8307 us, Tphi2为7.6607 us:
Coupler在CZ做门点，做门时间为27.6ns，coupler引入的错误为：0.0079%,其中白噪声引入的错误为：0.0066%，1/f噪声引入的错误为：0.0013%:


### Coupler热激发导致的dephasing


In [114]:
def DephasingCouplerThermal(phiSet):
    T1Ccoupler = 5e-6
    T_est = 60e-3
    phiIndex = np.argmin(abs(fluxList-phiSet))

    wc = freqList[3][phiIndex]*2*np.pi*1e9
    wq = (freqList[1][phiIndex]+freqList[2][phiIndex])/2*2*np.pi*1e9
    hbar=1.054560652926899e-34;     
    h = hbar*2*np.pi;     
    e = 1.60217662e-19;     
    phi0 = h/2/e
    kb = 1.380649e-23
    R = 50
    
    
    nc = 1/(np.exp(hbar*wc/kb/T_est)-1)
    chi = -(couplingQCList[phiIndex]*1e9*2*np.pi)**2/(wq-wc)*1/(1+(wq-wc)/(anharmonicityList[phiIndex]*1e9*2*np.pi))
    kb = 1.380649e-23;h = 6.62607015e-34;hbar = h/2/np.pi
    kappa = 1/T1Ccoupler

    eta = kappa**2/(kappa**2+4*chi**2)
    gammaPhi = eta*4*chi**2/kappa*nc
    Tphi = 1/gammaPhi
    errorPhi1 = 2*1/2*(tgate/1e9/(Tphi))
    print(nc,tgate,Tphi/1e-6,errorPhi1*100)




phiSet = offFlux
DephasingCouplerThermal(phiSet)
phiSet = coupling10Flux
DephasingCouplerThermal(phiSet)







0.016247841462152857 27.62135864009951 307.7341703185621 0.008975720379542598
0.015831246186698025 27.62135864009951 315.8312403132985 0.00874560686672403


## 浮地比特-接地coupler-浮地qubit 1V4系统 (比特间通过额外的电容进行耦合)

In [2]:
from QubitEnergyFineSimulation import FloatingTransmonWithGroundedCouplerAndCapacitor1V4

CJ=9.8e-15;     SRatio=6.0; 
C12Raw=4.076e-15
C12 = C12Raw+CJ
C23=20.632e-15
C24=20e-15
C34 = 80e-15

C35=C23
C56=C12Raw+CJ
C45 = C24

C1=157.21e-15-C12Raw-2*C23-2*C24
C2=158.87e-15-C12Raw-2*C23-2*C24
C3=168.67e-15+CJ-C23-C34-C35
C4=168.67e-15-C24-C34-C45
C5=C2
C6=C1
R=8500



Capa = np.array([
    [C1,    C12,    0,      0,      0,      0,      C23,    C24,    C23,    C24,    0,     0,    0,      0,      0,      0,     0,      0],
    [C12,   C2,     C23,    C24,    0,      0,      0,      0,      0,      0,      C23,   C24,  0,      0,      0,      0,     0,      0],
    [0,     C23,    C3,     C34,    C35,    0,      0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [0,     C24,    C34,    C4,     C45,    0,      0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [0,     0,      C35,    C45,    C5,     C56,    0,      0,      0,      0,      0,     0,    C35,    C45,    0,      0,     0,      0],
    [0,     0,      0,      0,      C56,    C6,     0,      0,      0,      0,      0,     0,    0,      0,      C35,    C45,   C35,    C45,],
    [C23,   0,      0,      0,      0,      0,      C3,     C34,    0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [C24,   0,      0,      0,      0,      0,      C34,    C4,     0,      0,      0,     0,    0,      0,      0,      0,     0,      0],
    [C23,   0,      0,      0,      0,      0,      0,      0,      C3,     C34,    0,     0,    0,      0,      0,      0,     0,      0],
    [C24,   0,      0,      0,      0,      0,      0,      0,      C34,    C4,     0,     0,    0,      0,      0,      0,     0,      0],
    [0,     C23,    0,      0,      0,      0,      0,      0,      0,      0,      C3,    C34,  0,      0,      0,      0,     0,      0],
    [0,     C24,    0,      0,      0,      0,      0,      0,      0,      0,      C34,   C4,   0,      0,      0,      0,     0,      0],
    [0,     0,      0,      0,      C35,    0,      0,      0,      0,      0,      0,     0,    C3  ,   C34,    0,      0,     0,      0],
    [0,     0,      0,      0,      C45,    0,      0,      0,      0,      0,      0,     0,    C34,    C4,     0,      0,     0,      0],
    [0,     0,      0,      0,      0,      C35,    0,      0,      0,      0,      0,     0,    0,      0,      C3,     C34,   0,      0],
    [0,     0,      0,      0,      0,      C45,    0,      0,      0,      0,      0,     0,    0,      0,      C34,    C4,    0,      0],
    [0,     0,      0,      0,      0,      C35,    0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     C3,     C34],
    [0,     0,      0,      0,      0,      C45,    0,      0,      0,      0,      0,     0,    0,      0,      0,      0,     C34,    C4],
])


Rlist = [R,R/SRatio,R,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio,R/SRatio]
para = [Capa,Rlist]
FTWC = FloatingTransmonWithGroundedCouplerAndCapacitor1V4(para)
el = FTWC.energyLevel

RlistWithoutC = [R,50,R,50,50,50,50,50,50]
paraWithoutC = [Capa,RlistWithoutC]
FTWithoutC = FloatingTransmonWithGroundedCouplerAndCapacitor1V4(paraWithoutC)
elWithoutC = FTWithoutC.energyLevel


couplerFrequency = el[FTWC.stateIndexList[3]]
qubitFreuqency = (el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]])/2
qubit02Freuqency = (el[FTWC.stateIndexList[5]]+el[FTWC.stateIndexList[6]])/2
coulingXYQCQ = -abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2 if FTWC.couplingMinus else abs(el[FTWC.stateIndexList[2]]-el[FTWC.stateIndexList[1]])/2
couplingXYQQDirect = -abs(elWithoutC[2]-elWithoutC[1])/2 if FTWithoutC.couplingMinus else abs(elWithoutC[2]-elWithoutC[1])/2
anharmonicity = (qubit02Freuqency-2*qubitFreuqency)
deltaFreqency = (qubitFreuqency-couplerFrequency)
couplingQC = np.sqrt((coulingXYQCQ-couplingXYQQDirect)*deltaFreqency)
coulingZZQCQ = abs(el[FTWC.stateIndexList[2]]+el[FTWC.stateIndexList[1]]-el[FTWC.stateIndexList[4]])
CouplerLeakage = FTWC.couplerLeakage

print(el[1:8])
print(elWithoutC[1:8])
print('Qubit Frequency:'+str(qubitFreuqency))
print('Coupler Frequency:'+str(couplerFrequency))
print('Anharmonicity:'+str(anharmonicity))
print('Qubit-Coupler-Qubit XY couling:'+str(coulingXYQCQ))
print('Qubit-Coupler-Qubit ZZ couling:'+str(coulingZZQCQ))
print('Qubit-Qubit direct couling:'+str(couplingXYQQDirect))
print('Qubit-Coupler couling:'+str(couplingQC))
print('CouplerLeakage:'+str(CouplerLeakage))

AttributeError: 'numpy.ndarray' object has no attribute 'dag'